In [1]:
import sys
import json
import ipywidgets as widgets
encoding = sys.getdefaultencoding()
import os
import pandas as pd

from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"
from IPython.display import display
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)

pd.options.display.float_format = '{:,.3f}'.format


In [2]:
list_files = os.listdir("./input")
mypath = Path().absolute()
folder = 'budget_opt\\'
img_folder = folder + 'img\\'

In [3]:
from BudgetOptimizer import Optimizer

from BudgetOptimizer import DateTimeID, dtype_, stat_name, regr_name, r2_stat_name, usecols_

## Read out dataset form CSV file

In [4]:
file_name = widgets.Dropdown(
    options = list_files,
    value=list_files[0],
    description='Source file:',
    disabled=False,
)
display(file_name)

Dropdown(description='Source file:', options=('4Camping.csv', '4home.csv', 'aliatic.csv', 'apotek.csv', 'audio…

In [13]:
df = pd.read_csv('input\\'+ file_name.value, sep="\t", infer_datetime_format=True,
                 usecols=usecols_, dtype=dtype_, parse_dates=DateTimeID)

df['periodStartDate'] = pd.to_datetime(df['periodStartDate'])
df.head(5)

,account,businessUnit,platformCode,channelCode,adCampaign,marketingInvestment,impressions,clicks,visits,conversions,deliveries,netRevenue,grossProfit,currencyCode,adGroup,periodStartDate
0,audiopartner,kytaryro,Compari,compari,NaN,0.000,0,0,63,2,1,"20,417.110","-1,144.320",CZK,(not set),2020-01-21
1,audiopartner,kytaryro,unpaid,direct,NaN,0.000,0,0,0,10,9,"58,887.130","14,298.720",CZK,(not set),2020-01-21
2,audiopartner,kytaryro,AdWords,googleadsdsother,NaN,0.000,0,0,2,0,0,0.000,0.000,CZK,(not set),2020-01-21
3,audiopartner,kytaryro,unpaid,organic,NaN,0.000,0,0,0,5,5,"5,961.500",735.000,CZK,(not set),2020-01-21
4,audiopartner,kytaryro,Compari,compari,-,198.180,0,106,0,0,0,0.000,0.000,CZK,NaN,2020-01-21


In [14]:
correlation_treshold=0.9
result, stat_ = Optimizer(df, 
                          alpha=0.2,
                          confidence_interval=2.0, 
                          change_investment = 0, 
                          level='platform',
                          correlation_treshold=correlation_treshold,
                          maximum_investment_change = 0.2)

Init estimator class
Load Settings
Load data
Correlation calculation executed
Optimizer class Initialization
Load optimizer settings
Load optimizer data


In [15]:
good_component_df = result[result['Corr_coeff']>=correlation_treshold]
good_component_df.reset_index(inplace=True, drop=True)
good_component_df.head(5)

,Business_unit,Client,Component,Corr_coeff,Invest,Invest_last_week,Invest_prev_week,Max_week_invest,Max_week_profit,Mean,Platform,Pr_value,Prof_last_week,Prof_prev_week,Profit,R_std,Regr_X,Regt_Y,X,Y,a,b,c,Down_line_Y,Up_line_Y,plus_invest,minus_invest,slope_plus,slope_minus,opt_invest,profit_change
0,kytarycz,audiopartner,kytarycz/ZboziCz,0.986,"54,641.640","2,958.190","3,020.260","6,943.050","35,936.960","17,220.958",,0.000,"13,856.880","17,001.380","281,420.870","5,414.463","[0.0, 694.3050000000001, 1388.6100000000001, 2...","[-861.5527323627441, 2095.9799627174943, 5316....","[664.57, 5312.91, 5506.129999999999, 5550.16, ...","[1620.07, 29003.29, 21366.649999999998, 31005....","-33,214.100","-8,139.500",-861.553,"[-11690.479648655239, -8732.946953575, -5512.0...","[9967.374183929749, 12924.906879009988, 16145....","3,624.312","2,416.208",6.139,-5.700,"3,624.312","3,708.208"
1,kytarypl,audiopartner,kytarypl/Ceneo,0.993,"125,900.130","8,224.650","9,773.570","14,767.630","28,536.940","11,780.693",,0.000,"9,665.110","10,808.100","193,557.050","10,980.448","[0.0, 1476.7630000000001, 2953.5260000000003, ...","[1623.1074666839932, 2219.4259342991672, 3086....","[1417.11, 10029.96, 11257.4, 12242.77999999999...","[1483.26, 9417.84, 15468.86, 16269.04, 17050.1...","-1,313.674","-3,945.700","1,623.107","[-20337.78939905622, -19741.470931441043, -188...","[23584.004332424203, 24180.32280003938, 25047....","11,728.284","7,818.856",5.130,-3.126,"11,728.284","10,027.892"
2,kytarycz,audiopartner,kytarycz/Sklik,0.902,"601,212.430","24,477.720","28,727.950","70,852.530","262,474.220","112,470.721",,0.000,"131,346.220","93,567.410","1,810,650.170","43,849.168","[0.0, 7085.253000000001, 14170.506000000001, 2...","[-25183.067663453192, 3410.3196470801813, 3088...","[6926.16, 49452.97, 58688.19, 55542.12, 65445....","[6251.589999999999, 145878.59, 144270.11, 2298...","728,051.549","176,840.095","-25,183.068","[-112881.40404482716, -84288.01673429378, -568...","[62515.268717920764, 91108.65602845413, 118579...","34,473.540","22,982.360",3.443,-3.557,"34,473.540","19,784.639"
3,kytarycz,audiopartner,kytarycz/HeurekaCz,0.916,"177,472.350","13,193.960","13,799.340","21,237.220","72,136.600","32,582.018",,0.000,"23,237.470","21,786.050","510,346.060","16,843.217","[0.0, 2123.722, 4247.444, 6371.166000000001, 8...","[4273.429773534856, 9941.728841806027, 15723.6...","[0.0, 11368.36, 12243.240000000005, 19624.0300...","[5324.07, 29688.25, 27035.970000000005, 66048....","-282,811.194","-107,018.241","4,273.430","[-29413.003449744527, -23744.704381473355, -17...","[37959.86299681424, 43628.16206508541, 49410.0...","16,559.208","11,039.472",3.045,-2.968,"16,559.208","8,405.019"
4,kytarycz,audiopartner,kytarycz/AdWords,1.000,"3,528,525.520","200,835.080","200,765.980","553,728.790","1,243,034.510","560,470.623",,0.000,"554,556.960","542,495.400","9,064,091.470","98,212.986","[0.0, 55372.87900000001, 110745.75800000002, 1...","[-30036.93595784496, 151756.70675774553, 31529...","[32261.24999999998, 207628.89999999997, 209732...","[76990.89000000001, 603410.47, 596641.3, 98766...","1,810,536.119","523,300.095","-30,036.936","[-226462.90699388483, -44669.264278294344, 118...","[166389.03507819492, 348182.6777937854, 511722...","240,919.176","160,612.784",2.269,-2.450,"204,685.392","9,205.201"


In [16]:
stat_

{'Invest_last_week': 0,
 'Controlled_Profit_last_week': 807398.5800000002,
 'Total_profit_last_week': 4525823.329999999,
 'Expected_profit_change': 24754.644568403113,
 'Controlled_investment': 5728830.42,
 'Non-controlled_investment': 3742405.81,
 'Controlled_profit': 13764916.209999999,
 'Non_controlled_profit': 6725905.85,
 'Confidence_interval': 2.0,
 'Unallocated_extra_budgets': 0.0,
 'Budget_pull': 66201.056}

In [17]:
component_list = good_component_df['Component'].to_list()
index_list = list(good_component_df.index)
component_menu = list(zip(component_list, index_list))

In [18]:
component = widgets.Dropdown(
    options=component_menu,
    value=0,
    description='Component:',
)
display(component)

Dropdown(description='Component:', options=(('kytarycz/ZboziCz', 0), ('kytarypl/Ceneo', 1), ('kytarycz/Sklik',…

In [26]:
weeks = 12 # TODO It need to be get from statistics dictionary
import plotly.graph_objects as go
fig = go.Figure()
    # Add traces
#x=np.linspace(0, zzz_df.Max_week_invest.to_list()[0], num=100)
fig.add_trace( go.Scatter(x = (good_component_df.loc[component.value]['X']) ,
               y = (good_component_df.loc[component.value]['Y']),
               mode = 'markers',
               name = f'Real data for last {weeks} weeks'))

fig.add_trace( go.Scatter(x = [(good_component_df.loc[component.value]['Invest_last_week']).tolist()],
               y = [(good_component_df.loc[component.value]['Prof_last_week']).tolist()],
               mode='markers',
               name='Last week'))

fig.add_trace( go.Scatter(x = [(good_component_df.loc[component.value]['Invest_prev_week']).tolist()],
               y = [(good_component_df.loc[component.value]['Prof_prev_week']).tolist()],
               mode='markers',
               name='Previous week'))

try:
    fig.add_trace( go.Scatter(x = good_component_df.loc[component.value]['Regr_X'] ,
               y = good_component_df.loc[component.value]['Regt_Y'],
               mode='lines',
               name='Regression'))

    fig.add_trace( go.Scatter(x = list(good_component_df.loc[component.value]['Regr_X']),
               y = list(good_component_df.loc[component.value]['Up_line_Y']),
               mode='lines',
               name='Upper confidence interval'))
    
    fig.add_trace( go.Scatter(x = list(good_component_df.loc[component.value]['Regr_X']),
               y = list(good_component_df.loc[component.value]['Down_line_Y']),
               mode='lines',
               name='Lower confidence interval'))
    
    fig.add_trace( go.Scatter(x = [(good_component_df.loc[component.value]['opt_invest']).tolist()],
               y = [good_component_df.loc[component.value]['Prof_last_week'] + good_component_df.loc[component.value]['profit_change']],
               mode='markers',
               name='Optimal point next week'))
except:
    pass

fig.update_layout( title_text= f"Paramiters of {component_menu[component.value][0]} componet are: {good_component_df.loc[component.value]['Corr_coeff']}corr. coeff ", 
                  title_font=dict(size=16))
fig.show()